# Übung 2.1 - Content Based Recommender - Ähnliche Filme

Die Vorschläge welche wir in der 1. Übung gemacht haben, sind noch nicht wirklich gut. Denn alle Personen erhalten die gleichen Vorschläge, unabhängig vom Geschmack der Person. Um bessere Vorschläge machen zu können, werden wir nun die Meta-Daten der Filme hinzuziehen. So können wir z.B. nachdem eine Person einen Film geschaut hat, ähnliche Filme vorschlagen. z.B. Filme mit einem ähnlichen Cast oder ähnlichen Beschreibung.

In [ ]:
import ast
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances

Einlesen der Film Metadaten

In [ ]:
list_columns = ['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages', 'cast', 'director', 'producer', 'writer', 'music']

movies = pd.read_csv('data/movies.csv', keep_default_na=False, converters={col: ast.literal_eval for col in list_columns})
movies = movies.set_index('title', drop=False)

In [ ]:
movies.head()

### Movie Description Based Recommender
Wir werden nun einen Recommender implementieren, welcher die Beschreibungen der Filme beachtet.\
Hinweis: [sklearn.feature_extraction.text.TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
# Transformiere die descriptions mit dem TFidfVectorizer zu einer tfidf-Matrix


Hinweis: [sklearn.feature_extraction.text.TfidfVectorizer.get_feature_names_out](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer.get_feature_names_out)

In [ ]:
# Um die tfidf_matrix besser zu verstehen können wir diese wieder in ein DataFrame umwandeln, 
# mit den Filmtitel als Zeilen (index) und Feature-Names in den Spalten (columns)


#### Cosine Similarity

Wir werden nun eine Ähnlichkeits-Matrix aller Filme berechnen. Verwende hierfür die Cosine-Similarity, welche folgendermassen definiert ist:

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||} $

Hinweis: [sklearn.metrics.pairwise.cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)

In [ ]:
# Berechne die Ähnlichkeits-Matrix


In [ ]:
# Wir können Ähnlichkeitsmatrix ebenfalls wieder in ein DataFrame umwandeln, 
# jetzt mit dem Film Titel im Index und auch in den Spalten


Da wir nun die paarweisen Ähnlichkeiten aller Filme haben, ist der nächste Schritt nun die 20 änlichsten Filme eines Filmes zu finden.

In [ ]:
# implementiere nun folgende Funktion die basierend auf einem Film-Titel die 20 ähnlichsten Filme zurückgibt
def get_recommendations(similarities, title):
    

Jetzt können wir für ein paar Filme die ähnlichste Filme ermitteln

In [ ]:
get_recommendations(text_similarities, 'The Dark Knight').head(30)

### Metadata Based Recommender

Der vorherige Recommender basiert nur auf den Beschreibungen der Filme. Ein grosser Anteil was einen Film ausmacht sind aber auch Schauspieler und der Direktor.
Darum werden wir nun einen Recommender bauen, welche diese Aspekte auch einbezieht.

Zuerst erstellen wir eine Funktion um aus einer Spalte *col* mit Listen ein One Hot Encoding zu erstellen
Dazu wählen wir die ersten *take_n* Elemente jedes Eintrag, erstellen ein One Hot Encoding der Elemente welche mindestens *min_occurence* auftreten

In [ ]:
# Wir verkleinern das Dataset etwas um die Berechnungen etwas zu beschleunigen
movies_small = movies[movies.vote_count > 1000]
movies_small.shape

In [ ]:
# Betrachten wir die Spalte mit den Schauspielern
col = movies_small.cast
take_n = 3
min_occurence = 2
col

Hinweis: auf einer pandas.Series welche aus Listen besteht, können auch die Stringfuktionen angewendet werden\
Ersten n Elemente auswählen: [pandas.Series.str.slice](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html)\
Elemente der Liste zusammenführen: [pandas.Series.str.join](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.join.html)\
Onehot-Encoding eines Strings: [pandas.Series.str.get_dummies](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.get_dummies.html)

In [ ]:
# Selectieren der ersten take_n Elemente 


In [ ]:
# Zusammenführen der Elemente der Listen, mit '|' getrennt (da get_dummies default-Seperator | ist) 


In [ ]:
# Onehot-Endcoding mit get_dummies()


Hinweis: [pandas.DataFrame.sum](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html)

In [ ]:
# Filtern der Spalten, mit weniger als min_occurence Elementen


Für nun die einzelnen Schritte von oben, in folgender Funktion zusammen, um für eine beliebige Spalte ein Onehot-Encoding zu machen

In [ ]:
# One Hot Encoder
def one_hot_encoder(col, take_n=3, min_occurence=2):
    

Berechne nun mit one_hot_encoder verschiedene Splaten und führe sie in ein grosses features-DataFrame zusammen\
Hinweis: [pandas.concat](https://pandas.pydata.org/docs/reference/api/pandas.concat.html)

In [ ]:
# Erstelle meta_features-DataFrame


In [ ]:
# Schaue die Features für einen Film an, welche auf 1 gesetzt sind
meta_features.loc['The Dark Knight'].sort_values(ascending=False).head(30)

Nun können wir eine neue Ählichkeits-Matrix berechnen mit unseren Meta-Features und z.B. der Jaccard-Ähnlichkeit\
Hinweis: [sklearn.metrics.pairwise_distances](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html)

In [ ]:
# neue Ähnlichkeits-Matrix berechnen


Jetzt können wir, wie zuvor mit der neuen Ähnlichkeitsmatrix wieder Vorschläge generieren

In [ ]:
get_recommendations(meta_similarities_jaccard, 'The Dark Knight').head(10)